In [ ]:
import os
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
imagesPath = '/kaggle/input/utkface-new/UTKFace/'
allImages = os.listdir(imagesPath)
totalNbrOfImages = len(allImages)
print("Total count of images : ", totalNbrOfImages)

In [ ]:
# Creating the Image, Ages and the Genders Array:
updatedImages = []
allAgeNbrs = []
allGenders = []
counter = 0

for file in allImages:
    #Use this if condition to restrict input data to 30 images
    #if (counter >= 30):
    #    break

    image = cv2.imread(imagesPath+file,0)
    image = cv2.resize(image,dsize=(200,200))
    image = image.reshape((image.shape[0],image.shape[1],1))
    updatedImages.append(image)
    split_var = file.split('_')
    allAgeNbrs.append(split_var[0])
    allGenders.append(int(split_var[1]))
    
    #counter += 1
    #print("allAgeNbrs : ", allAgeNbrs)
    #print("allGenders : ", allGenders)

In [ ]:
# Defining the function to display Images:

def displayImage(img):
    plt.imshow(img[:,:,0])
    plt.set_cmap('gray')
    plt.show()
    
# Function for defining the Age Groups:

def age_group(age):
    if age >=0 and age < 18:
        return 1
    elif age < 30:
        return 2
    elif age < 80:
        return 3
    else:
        return 4

In [ ]:
## Displaying an Image from the Dataset:
imageCtr = 5
sampleImage = updatedImages[imageCtr]
print("Gender:",allGenders[imageCtr],"Age:",allAgeNbrs[imageCtr])
displayImage(sampleImage)

In [ ]:
# Pre-processing:

labels = np.zeros((totalNbrOfImages,2),dtype='float32')
features = np.zeros((totalNbrOfImages,sampleImage.shape[0],sampleImage.shape[1],1),dtype = 'float32')

for i in range(totalNbrOfImages):
    labels[i,0] = age_group(int(allAgeNbrs[i])) / 4
    labels[i,1] = int(allGenders[i])
    features[i] = updatedImages[i]

features = features / 255

displayImage(features[15])

Testing markdown..........

In [ ]:
# Splitting the Original Dataset into Training and Testing Dataset:

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2,shuffle  = True)
print("Samples in Training:",x_train.shape[0])
print("Samples in Testing:",x_test.shape[0])

In [ ]:
# Defining the Neural Network Layers:

inputLayer = Input(shape=(200,200,1))
convLayer1 = Conv2D(32, kernel_size=(3, 3),activation='relu')(inputLayer)
convLayer2 = Conv2D(64, kernel_size=(3, 3),activation='relu')(convLayer1)
maxPoolLayer1 = MaxPooling2D(pool_size=(2, 2))(convLayer2)
convLayer3 = Conv2D(128, kernel_size=(3, 3),activation='relu')(maxPoolLayer1)
maxPoolLayer2 = MaxPooling2D(pool_size=(2, 2))(convLayer3)
dropoutLayer = Dropout(0.25)(maxPoolLayer2)
flattenlayer = Flatten()(dropoutLayer)

#defining layers for age modeling 
dropout = Dropout(0.5)
age_model = Dense(128, activation='relu')(flattenlayer)
age_model = dropout(age_model)
age_model = Dense(64, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(32, activation='relu')(age_model)
age_model = dropout(age_model)
age_model = Dense(1, activation='relu')(age_model)

#defining layers for gender modeling 
dropout = Dropout(0.5)
gender_model = Dense(128, activation='relu')(flattenlayer)
gender_model = dropout(gender_model)
gender_model = Dense(64, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(32, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(16, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(8, activation='relu')(gender_model)
gender_model = dropout(gender_model)
gender_model = Dense(1, activation='sigmoid')(gender_model)

In [ ]:
# Compile the Model:

CNNModel = Model(inputs=inputLayer, outputs=[age_model,gender_model])
CNNModel.compile(
    optimizer = 'adam', 
    loss =['mse','binary_crossentropy'],
    metrics=['mean_squared_error', 'binary_accuracy']
)

CNNModel.summary()

In [ ]:
# Train the Model:

h = CNNModel.fit(
    x_train,
    [y_train[:, 0], y_train[:, 1]],  
    validation_data=(x_test, [y_test[:, 0], y_test[:, 1]]),
    epochs=40,
    batch_size=128,
    shuffle=True 
)

In [ ]:
CNNModel.save('nalin_model_pretrain.h5')

In [ ]:
# Defining the functions for getting the Predictions:

def get_age(distr):
    distr = distr*4
    if distr >= 0.65 and distr <= 1.4:return "0-18"
    if distr >= 1.65 and distr <= 2.4:return "19-30"
    if distr >= 2.65 and distr <= 3.4:return "31-80"
    if distr >= 3.65 and distr <= 4.4:return "80 +"
    return "Unknown"

def get_gender(prob):
    if prob < 0.5:return "Male"
    else: return "Female"

def get_result(sample):
    sample = sample/255
    val = CNNModel.predict( np.array([ sample ]) )
    age = get_age(val[0])
    gender = get_gender(val[1])
    print("Values:",val,"\nPredicted Gender:",gender,"Predicted Age:",age)

In [ ]:
# Taking the predictions for a set of sample data points:

indexes = [2520,5140,8005,12010]
for idx in indexes:
    sample = updatedImages[idx]
    displayImage(sample)
    print("Actual Gender:",get_gender(allGenders[idx]),"Age:",allAgeNbrs[idx])
    res = get_result(sample)
    
    
myImagespath = '/kaggle/input/my-photos-1/'
myImages = os.listdir(myImagespath)
for myImage in myImages:

    image = cv2.imread(myImagespath+myImage,0)
    image = cv2.resize(image,dsize=(200,200))
    image = image.reshape((image.shape[0],image.shape[1],1))
    #updatedMyImages.append(image)
    displayImage(image)
    res = get_result(image)